# 요구사항
1. 우리나라 대기업 국내외 관련기사->투자, 협약 맺음, 건설 내용잇음더 좋음
2. 중국기업 관련기사 국내외
3. 중국 기업이 우리나라에 투자하거나 우리나라 기업과 협력 관련기사->거의 안뜸
4. 중국 기업이 우리나라에 들어오는 기사->요즘은 알리,테무가 많이 뜨더라
이정도..?

# pip install & import & setting

In [191]:
# !pip3 install requests beautifulsoup4
# !pip3 install pandas
# !pip3 install openpyxl


In [192]:
import re
import requests
from bs4 import BeautifulSoup
import pandas as pd

from datetime import datetime
import os

In [193]:
pd.set_option('display.max_colwidth', 100)

# Common Functions & Variables

In [194]:
current_time = datetime.now().strftime('%y%m%d%H%M%S')

In [195]:
def get_bs4_html(url: str):
    response = requests.get(url)
    if not response.status_code == 200:
        print("Failed to retrieve the webpage")
        return

    html_content = response.content
    soup = BeautifulSoup(html_content, 'html.parser')
    return soup

# Naver Crawling

## `https://news.naver.com` setting

In [196]:
naver_news_url = "https://news.naver.com/"

## 속보 크롤링

In [197]:
# 속보
def newsflash_crawling(root_html: BeautifulSoup):
    newsflash_list = root_html.find('div', class_="cjs_nf_list _item_list").find_all('a')

    if not newsflash_list:
        print('속보를 찾을 수 없습니다.')
    else:
        newsflash_data = []
        for newsflash in newsflash_list:
            href = newsflash.get('href')
            title = newsflash.find('h4', class_="cn_title").text.strip()
            company_name = newsflash.find('div', class_="cn_name").text.strip()
            newsflash_data.append({'topic':'속보', 'title': title, 'company_name': company_name, 'link': href})
        
        df = pd.DataFrame(newsflash_data)
        return df

df_newsflash = newsflash_crawling(get_bs4_html(naver_news_url))
df_newsflash

,topic,title,company_name,link
0,속보,지역구 개표율 65% 돌파…민주 172·국힘 112·조국 11·개혁 2·새미래 1 전망,데일리안,https://n.news.naver.com/article/119/0002819086
1,속보,충남 성일종 당선 등 일부 후보 유력·확실 '윤곽',대전일보,https://n.news.naver.com/article/656/0000086377
2,속보,춘천·철원·화천·양구 을 한기호 당선… 4선 성공,강원도민일보,https://n.news.naver.com/article/654/0000071395
3,속보,"서울 용산, 권영세 당선 유력",한국경제,https://n.news.naver.com/article/015/0004971153
4,속보,"대통령실 위치한 서울 용산, 권영세 '당선 유력'…득표율 53.32%",서울경제,https://n.news.naver.com/article/011/0004326012
5,속보,"서울 중·성동갑, 더불어민주당 전현희 당선 확실",이데일리,https://n.news.naver.com/article/018/0005711661
6,속보,"4·10총선 개표율 50% 현재 민주 153곳, 국민의힘 97곳 1위",강원일보,https://n.news.naver.com/article/087/0001037411


## 섹션 크롤링

In [198]:
# TODO: 섹션 딕셔너리 만들기
def get_section_dictionary():
  
  return

# section map
section_dictionary = {
    101: {
        'main_section': "경제",
        'sub_section': {
          259: "금융",
          258: "증권",
            261: "산업/재계",
            771: "중기/벤처",
            260: "부동산",
            262: "글로벌 경제",
            310: "생활경제",
            263: "경제 일반"
        }
    },
    104: {
        'main_section': "세계",
        'sub_section': {
          231: "아시아/호주"
        }
    }
}

In [199]:
def section_crawling(root_html: BeautifulSoup, section_name: str):
    section_list = root_html.find('div', id="newsct").find_all('li')

    if not section_list:
        print(f'{section_name} 섹션을 찾을 수 없습니다.')
    else:
        section_data = []

        for section in section_list:
            content_html = section.find('div', class_="sa_text")
            if not content_html:
                continue
            href = content_html.find('a', class_="sa_text_title").get('href')
            title = content_html.find('strong', class_="sa_text_strong").text.strip()
            description = content_html.find('div', class_="sa_text_lede").text.strip()
            company_name = content_html.find('div', class_="sa_text_press").text.strip()
            
            section_data.append({'topic': section_name, 'title': title, 'company_name': company_name, "description": description,'link': href})
        
        df = pd.DataFrame(section_data)
        return df

In [200]:
all_section_df_list = []

for main_section_key, main_section_value in section_dictionary.items():
	main_section_html = get_bs4_html(f"https://news.naver.com/section/{main_section_key}")
	main_section_df = section_crawling(main_section_html, main_section_value['main_section'])

	if main_section_df is not None:
		print(f"{main_section_value['main_section']} 크롤링 성공")
		all_section_df_list.append(main_section_df)

	if main_section_value['sub_section']:
		for sub_section_key, sub_section_value in main_section_value['sub_section'].items():
			sub_section_html = get_bs4_html(f"https://news.naver.com/breakingnews/section/{main_section_key}/{sub_section_key}")
			sub_section_df = section_crawling(sub_section_html, sub_section_value)

			if main_section_df is not None:
				print(f"{main_section_value['main_section']} - {sub_section_value} 크롤링 성공")
				all_section_df_list.append(sub_section_df)

all_section_df = pd.concat(all_section_df_list, ignore_index=True)

경제 크롤링 성공
경제 - 금융 크롤링 성공
경제 - 증권 크롤링 성공
경제 - 산업/재계 크롤링 성공
경제 - 중기/벤처 크롤링 성공
경제 - 부동산 크롤링 성공
경제 - 글로벌 경제 크롤링 성공
경제 - 생활경제 크롤링 성공
경제 - 경제 일반 크롤링 성공
세계 크롤링 성공
세계 - 아시아/호주 크롤링 성공


## join df & to xlsx & to csv

In [201]:
total_news_df = pd.concat([df_newsflash, all_section_df])
total_news_df

,topic,title,company_name,link,description
0,속보,지역구 개표율 65% 돌파…민주 172·국힘 112·조국 11·개혁 2·새미래 1 전망,데일리안,https://n.news.naver.com/article/119/0002819086,NaN
1,속보,충남 성일종 당선 등 일부 후보 유력·확실 '윤곽',대전일보,https://n.news.naver.com/article/656/0000086377,NaN
2,속보,춘천·철원·화천·양구 을 한기호 당선… 4선 성공,강원도민일보,https://n.news.naver.com/article/654/0000071395,NaN
3,속보,"서울 용산, 권영세 당선 유력",한국경제,https://n.news.naver.com/article/015/0004971153,NaN
4,속보,"대통령실 위치한 서울 용산, 권영세 '당선 유력'…득표율 53.32%",서울경제,https://n.news.naver.com/article/011/0004326012,NaN
...,...,...,...,...,...
410,아시아/호주,"시진핑, 마잉주에 ""어떤 외세도 가족·국가 재결합 못막아""",아시아경제,https://n.news.naver.com/mnews/article/277/0005403994,"시진핑 중국 국가주석이 중국을 방문 중인 마잉주 전 대만 총통을 만나 ""해협 양안의 동포들은 모두 유구하고 찬란한 역사를 가진 중화문명을 창조한 중화민족에 속한다""고 말한 ..."
411,아시아/호주,"필리핀 대통령 ""미일정상과 남중국해 안보 합의할 것""…中, 반발(종합2보)",연합뉴스,https://n.news.naver.com/mnews/article/001/0014623288,"11일 3국 정상회의…""항행의 자유 유지 등 협력 이행 구체적 방안 논의"" 中 외교부 ""평화 명분으로 근육 과시·난국 조성…명백한 패권행위"" 강종훈 특파원 홍제성 기자 = ..."
412,아시아/호주,"필리핀 대통령 ""美·日과 정상회담서 남중국해 평화 이행 논의""",뉴시스,https://n.news.naver.com/mnews/article/003/0012484120,오는 11일로 예정된 미국·일본·필리핀 삼자 정상회담에서 남중국해 문제가 중점 논의될 전망이다. 알자지라 등에 따르면 페르디난드 마르코스 필리핀 대통령은 10일(현지시각) ...
413,아시아/호주,"시진핑, 마잉주에 ""어떤 외세도 가족·국가 재결합 못막아""(종합)",연합뉴스,https://n.news.naver.com/mnews/article/001/0014623277,"마잉주 ""양안 전쟁 감당못할 부담…분쟁 평화적 처리해야"" 美日 정상회담 당일 시진핑 '친중' 마잉주 前대만총통 접견 맞불 정성조 특파원 홍제성 기자 = 시진핑 중국 국가주석..."


In [202]:
# to csv
total_news_df.to_csv(f'naver_{current_time}.csv', index=False)
print(f'naver_{current_time}.csv 파일 생성')

# to xlsx
file_name = 'naver.xlsx'

if os.path.isfile(file_name):
    with pd.ExcelWriter(file_name, mode='a') as writer:
        total_news_df.to_excel(writer, sheet_name=current_time, index=False)
        print(f'naver.xlsx 파일 {current_time} 시트 생성')
else:
    with pd.ExcelWriter(file_name) as writer:
        total_news_df.to_excel(writer, sheet_name=current_time, index=False)
        print(f'naver.xlsx 파일 생성')

naver_240410235239.csv 파일 생성
naver.xlsx 파일 생성


# kr.investing.com Crawling